In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

DIFF_RATING_OUTPUT_NAME = 'diff_rating_pred_data.csv'
SAME_RATING_OUTPUT_NAME = 'same_rating_pred_data.csv'
cluster_types = ['tag','word2vec_cluster','bpr_cluster']

<h1>Load and Filter Data</h2>

In [2]:
#Load problem metadata
problems = pd.read_csv("../../ChessPuzzleEmbeddings/bpr_puzzle_meta.tsv",sep='\t')

In [3]:
#Read the attempts dataset and filter out users that have played less than 1k games
attempts = pd.read_csv("/w/225/1/chess/tactics/glicko_user_tactics_problem.csv_00")
attempts = attempts.loc[attempts['userGamesPlayed'] >= 1000]

In [4]:
#Assign rating labels to problems in metadata so that problems 
#with ratings within the same interval have the same label
start = 375
end = 3025
interval = 25
problems['rating_labels'] = pd.cut(problems['rating'],np.arange(start,end,interval),labels=range(1,((end-start)//interval)))
#Merge problem metadata to join cluster and rating labels to the attempts by puzzle ID
attempts=pd.merge(attempts,problems[['tactics_problem_id','word2vec_cluster','tag','bpr_cluster','rating_labels','rating']],on='tactics_problem_id', how='inner')

In [5]:
#Filter the data down to 8 columns 
attempts = attempts[['user_hash','tactics_problem_id','word2vec_cluster','bpr_cluster','tag','ratingUser','rating_labels','is_passed','rating']]

In [10]:
attempts.shape

(48635761, 9)

In [11]:
def join_cluster_attempt_count(cluster_type):
    '''
    Given a cluster type (tag, word2vec, bpr), add
    a new column to the attempts data with the number
    of times the user in each row has attempted the cluster_type
    the puzzle belongs to
    '''
    print('Joining attempts with cluster attempt count for: ' + cluster_type)
    user_count_list = attempts.groupby(['user_hash',cluster_type])['tactics_problem_id'].count()
    user_count_list = user_count_list.rename(cluster_type + '_attempt_count')
    return pd.merge(attempts,user_count_list,on=['user_hash',cluster_type],how='left')

#Create lists of cluster counts for each user
for cluster in cluster_types:
    attempts = join_cluster_attempt_count(cluster)

Joining attempts with cluster attempt count for: tag
Joining attempts with cluster attempt count for: word2vec_cluster
Joining attempts with cluster attempt count for: bpr_cluster


In [13]:
print(attempts.shape)
attempts.head()

(48635761, 12)


,user_hash,tactics_problem_id,word2vec_cluster,bpr_cluster,tag,ratingUser,rating_labels,is_passed,rating,tag_attempt_count,word2vec_cluster_attempt_count,bpr_cluster_attempt_count
0,3da90302d3d94216f2c06cc49efb2b122bee379c814392...,30668,4,1,Vulnerable King,1829,73,0,2197,310,1191,777
1,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,30668,4,1,Vulnerable King,1775,73,1,2197,269,1336,925
2,7a348494ef7ba27a6fe2ed089ac84aca4a95ba77b95484...,30668,4,1,Vulnerable King,1872,73,0,2197,248,1000,707
3,9adee4e89a43a5f412bb4227433937655e99a11426c109...,30668,4,1,Vulnerable King,1584,73,0,2197,104,867,635
4,709c6b9629f82206e2d0a980c2100253329e6c162a0c1b...,30668,4,1,Vulnerable King,1639,73,0,2197,31,84,54


In [15]:
#Gives 11346 unique users
x = 100
all_pass = attempts.loc[(attempts.tag_attempt_count >= x) & (attempts.word2vec_cluster_attempt_count >= x) & (attempts.bpr_cluster_attempt_count >= x)]
all_pass.reset_index(inplace=True)

In [16]:
print(all_pass.shape)

(26729792, 13)


<h2>Split Data for Pairing Win/Losses</h2>

In [17]:
#Create a table of all losses and all wins
loss_rows = all_pass.loc[all_pass.is_passed == 0]
win_rows = all_pass[~all_pass.index.isin(loss_rows.index)]

In [20]:
def rename_cols(dataframe,idee):
    '''
    Function that renames the column names in 
    the given dataframe by appending an identifier. 
    This distinguishes between the passed and failed puzzle
    '''
    new_names = []
    for col in dataframe.columns:
        new_names.append(col + '_' + idee)
    dataframe.columns = new_names
        
#Rename the dataframes
rename_cols(loss_rows,'2')
rename_cols(win_rows,'1')

In [21]:
#Create a running_count column of user attempts to keep track of the number of times
#the user appears in the dataset
loss_rows['user_attempt_number'] = loss_rows.groupby('user_hash_2').cumcount()+1
win_rows['user_attempt_number'] = win_rows.groupby('user_hash_1').cumcount()+1

/h/224/evanb/anaconda3/envs/chess_tactic/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/h/224/evanb/anaconda3/envs/chess_tactic/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [22]:
#Get only the first k wins and k losses from each user so that joining 
#doesnt create a HUGE data table
k = 100
first_100_wins = win_rows.loc[win_rows.user_attempt_number <= k]
first_100_losses = loss_rows.loc[loss_rows.user_attempt_number <= k]

<h2>Recombining Wins and Losses into Prediction Set</h2>

In [23]:
#Merge the wins and losses
combined_pairs = pd.merge(first_100_wins,first_100_losses,left_on=['rating_labels_1','user_hash_1'],
                    right_on=['rating_labels_2','user_hash_2'],how='left')

In [24]:
#Filter pairs so that only rows where each cluster_type has 2 different values
#Gives: 1,631,988 rows
diff_pairs = combined_pairs.loc[(combined_pairs.tag_1 != combined_pairs.tag_2) & 
                                (combined_pairs.word2vec_cluster_1 != combined_pairs.word2vec_cluster_2) & 
                                (combined_pairs.bpr_cluster_1 != combined_pairs.bpr_cluster_2)]

In [82]:
#Remove Unnecessary columns
diff_pairs = diff_pairs.drop(['user_hash_2','ratingUser_2','user_attempt_number_x','is_passed_1','is_passed_2','user_attempt_number_y','index_1','index_2','rating_labels_1','rating_labels_2','tag_attempt_count_2', 'word2vec_cluster_attempt_count_2','bpr_cluster_attempt_count_2'],axis=1)
diff_pairs = diff_pairs.rename(columns={'user_hash_1':'user_hash','ratingUser_1':'ratingUser','tactics_problem_id_1':'puzzle_1','tactics_problem_id_2':'puzzle_2'})

In [25]:
original_indices = diff_pairs[['index_1','index_2']]
original_indices = original_indices['index_1'].append(original_indices['index_2'])
print(original_indices.shape)

(3263976,)


<h2>Remove the attempts in the prediction set from the main attempts data</h2>

In [26]:
#Remove the indices for both attempts in each row of diff_pairs from the original attempts
attempts = attempts[~attempts.index.isin(original_indices)]

In [28]:
attempts.head()

,user_hash,tactics_problem_id,word2vec_cluster,bpr_cluster,tag,ratingUser,rating_labels,is_passed,rating,tag_attempt_count,word2vec_cluster_attempt_count,bpr_cluster_attempt_count
3,9adee4e89a43a5f412bb4227433937655e99a11426c109...,30668,4,1,Vulnerable King,1584,73,0,2197,104,867,635
4,709c6b9629f82206e2d0a980c2100253329e6c162a0c1b...,30668,4,1,Vulnerable King,1639,73,0,2197,31,84,54
7,e3390301248720198f4f2b4114c3cc61727568bd391e3b...,30668,4,1,Vulnerable King,1586,73,0,2197,86,265,183
10,0575612356f922da19918aa0b69d42a19886aec9d5aa71...,30668,4,1,Vulnerable King,1770,73,0,2197,76,190,132
11,5d69a4dc96b0a77bf602c085da3e976df736ac74a0601c...,30668,4,1,Vulnerable King,1879,73,0,2197,152,108,61


<h2>Get User Performance in Across Cluster Types</h2>

In [36]:
#Calculate performance rating per cluster per user
import time

def time_command(command,criteria):
    print('Running: ' + command)
    start = time.time()
    exec(command)
    print('Took: '  + str(time.time()-start))

def calc_user_performance(criteria,time=False):
    commands = ["global opponent_ratings_total; opponent_ratings_total = attempts.groupby(['user_hash',criteria])['rating'].sum()",
                "global num_cluster_games; num_cluster_games = attempts.groupby(['user_hash',criteria])['user_hash'].count()",
                "global wins_plus_losses; wins_plus_losses = attempts.groupby(['user_hash',criteria])['is_passed'].sum()"]
    for command in commands:
        if time:
            time_command(command,criteria)
        else:
            exec(command)
    return (opponent_ratings_total + 400*wins_plus_losses)/num_cluster_games

In [41]:
#Get user performance in each word2vec cluster, bpr cluster, and tag
word2vec_performance_user = calc_user_performance(criteria='word2vec_cluster',time=True)
word2vec_cluster_variance = word2vec_performance_user.var(level=0).agg('mean')
print(word2vec_cluster_variance)

bpr_performance_user = calc_user_performance(criteria='bpr_cluster',time=True)
avg_bpr_variance = bpr_performance_user.var(level=0).agg('mean')
print(avg_bpr_variance)

tag_performance_user = calc_user_performance(criteria='tag',time=True)
avg_tag_variance = tag_performance_user.var(level=0).agg('mean')
print(avg_tag_variance)

Running: global opponent_ratings_total; opponent_ratings_total = attempts.groupby(['user_hash',criteria])['rating'].sum()
Took: 67.50642132759094
Running: global num_cluster_games; num_cluster_games = attempts.groupby(['user_hash',criteria])['user_hash'].count()
Took: 85.62599921226501
Running: global wins_plus_losses; wins_plus_losses = attempts.groupby(['user_hash',criteria])['is_passed'].sum()
Took: 96.69156980514526
14228.356630596509


In [40]:
attempts.head()

,user_hash,tactics_problem_id,word2vec_cluster,bpr_cluster,tag,ratingUser,rating_labels,is_passed,rating,tag_attempt_count,word2vec_cluster_attempt_count,bpr_cluster_attempt_count
3,9adee4e89a43a5f412bb4227433937655e99a11426c109...,30668,4,1,Vulnerable King,1584,73,0,2197,104,867,635
4,709c6b9629f82206e2d0a980c2100253329e6c162a0c1b...,30668,4,1,Vulnerable King,1639,73,0,2197,31,84,54
7,e3390301248720198f4f2b4114c3cc61727568bd391e3b...,30668,4,1,Vulnerable King,1586,73,0,2197,86,265,183
10,0575612356f922da19918aa0b69d42a19886aec9d5aa71...,30668,4,1,Vulnerable King,1770,73,0,2197,76,190,132
11,5d69a4dc96b0a77bf602c085da3e976df736ac74a0601c...,30668,4,1,Vulnerable King,1879,73,0,2197,152,108,61


In [156]:
#Create frames from series
table_1 = word2vec_performance_user.to_frame().rename(columns={0:'word2vec_cluster_perf'})
table_1.reset_index(inplace=True)
table_2 = tag_performance_user.to_frame().rename(columns={0:'tag_perf'})
table_2.reset_index(inplace=True)
table_3 = bpr_performance_user.to_frame().rename(columns={0:'bpr_cluster_perf'})
table_3.reset_index(inplace=True)

In [150]:
diff_pairs.head()

,user_hash,puzzle_1,word2vec_cluster_1,bpr_cluster_1,tag_1,ratingUser,rating_1,tag_attempt_count_1,word2vec_cluster_attempt_count_1,bpr_cluster_attempt_count_1,puzzle_2,word2vec_cluster_2,bpr_cluster_2,tag_2,rating_2
0,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,30668,4,1,Vulnerable King,1775,2197,269,1336,925,30854.0,5.0,9.0,Trapped Piece,2176.0
1,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,28433,4,1,X-Ray Attack,1815,2278,151,1336,925,30555.0,5.0,9.0,Simplification,2286.0
2,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,28433,4,1,X-Ray Attack,1815,2278,151,1336,925,28407.0,5.0,9.0,Interference,2287.0
3,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,28107,4,1,Clearance Sacrifice,1795,2259,218,1336,925,30484.0,5.0,9.0,Exchange Sacrifice,2271.0
4,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,31854,2,6,Pawn Promotion,1820,2069,199,1332,1192,28848.0,5.0,9.0,Attacking Castled King,2072.0


In [157]:
#Admittedly horrible code to merge the performance data onto the pairs dataset
def merge_performance(cluster_type,table):
    table.rename(columns={cluster_type:cluster_type+'_1',cluster_type+'_perf':cluster_type+'_perf_1'},inplace=True)
    temp = pd.merge(diff_pairs,table[['user_hash',cluster_type+'_1',cluster_type+'_perf_1']],on=['user_hash',cluster_type+'_1'],how='inner')
    table.rename(columns={cluster_type+'_1':cluster_type+'_2',cluster_type+'_perf_1':cluster_type+'_perf_2'},inplace=True)
    return pd.merge(temp,table[['user_hash',cluster_type+'_2',cluster_type+'_perf_2']],on=['user_hash',cluster_type+'_2'],how='inner')
    
diff_pairs = merge_performance('word2vec_cluster',table_1)
diff_pairs = merge_performance('tag',table_2)
diff_pairs = merge_performance('bpr_cluster',table_3)

In [163]:
#diff_pairs = diff_pairs.drop(['cluster_score_1','cluster_score_2','tag_score_x','tag_score_y','bpr_score_x','bpr_score_y'],axis=1)
diff_pairs['passed_puzzle'] = np.full(diff_pairs.shape[0],0)

In [164]:
diff_pairs.head()

,user_hash,puzzle_1,word2vec_cluster_1,bpr_cluster_1,tag_1,ratingUser,rating_1,tag_attempt_count_1,word2vec_cluster_attempt_count_1,bpr_cluster_attempt_count_1,...,bpr_cluster_2,tag_2,rating_2,word2vec_cluster_perf_1,word2vec_cluster_perf_2,tag_perf_1,tag_perf_2,bpr_cluster_perf_1,bpr_cluster_perf_2,passed_puzzle
0,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,30668,4,1,Vulnerable King,1775,2197,269,1336,925,...,9.0,Trapped Piece,2176.0,2460.694047,2248.429544,2266.717557,2167.961672,2478.784934,2306.391089,0
1,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,28433,4,1,X-Ray Attack,1815,2278,151,1336,925,...,9.0,Simplification,2286.0,2460.694047,2248.429544,2277.657534,2263.691630,2478.784934,2306.391089,0
2,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,28433,4,1,X-Ray Attack,1815,2278,151,1336,925,...,9.0,Interference,2287.0,2460.694047,2248.429544,2277.657534,2346.801242,2478.784934,2306.391089,0
3,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,28107,4,1,Clearance Sacrifice,1795,2259,218,1336,925,...,9.0,Exchange Sacrifice,2271.0,2460.694047,2248.429544,2309.111628,2275.500000,2478.784934,2306.391089,0
4,b7654c69d380e16dc133d59936f34b1318e353faa450fa...,31854,2,6,Pawn Promotion,1820,2069,199,1332,1192,...,9.0,Attacking Castled King,2072.0,2165.770817,2248.429544,2194.860104,2294.595122,2092.504281,2306.391089,0


<h2>Clean Data and prepare for Prediction Task</h2>

In [177]:
#Randomly flip the rows so that if the passed_puzzle = 0, the puzzle_1 and corresponding fields
#belong to the passsed puzzle
def flip(row,field):
    name_1 = field  + '_1'
    name_2 = field + '_2'
    temp = row[name_2]
    row[name_2] = row[name_1]
    row[name_1] = temp
    return row
    

def flip_fields(row):
    row = flip(row,'word2vec_cluster')
    row = flip(row,'word2vec_cluster_perf')
    row = flip(row,'bpr_cluster')
    row = flip(row,'bpr_cluster_perf')
    row = flip(row,'tag')
    row = flip(row,'tag_perf')
    row = flip(row,'rating')
    row = flip(row,'puzzle')
    return row

def switch_win(row):
    num = random.uniform(0,1)
    if num > 0.5:
        row['passed_puzzle'] = 1
        row = flip_fields(row)
    return row
        
diff_pairs = diff_pairs.progress_apply(lambda x: switch_win(x),axis=1)

In [186]:
#Save the prediction dataset
diff_pairs.to_csv('../prediction_data/improved_prediction_task.csv',index=None,header=True)

<h2>Simple Prediction</h2>

In [194]:
random_samples = np.random.choice(diff_pairs.index,100000)
X = diff_pairs.iloc[random_samples].drop('user_hash',axis=1)
y = diff_pairs.iloc[random_samples]['passed_puzzle']

In [201]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
features = ['tag_perf','word2vec_cluster_perf','bpr_cluster_perf']

In [207]:
for feature in features:
    log_reg = LogisticRegression(solver='lbfgs')
    log_reg.fit(X[[feature+ '_1',feature +'_2']],y)
    scores = cross_val_score(log_reg,X[[feature+ '_1',feature +'_2']],y,cv=5,scoring='accuracy')
    #print(scores)
    print('Average accuracy over 5 folds for {}: {}'.format(feature,sum(scores)/len(scores)))

Average accuracy over 5 folds for tag_perf: 0.50859
Average accuracy over 5 folds for word2vec_cluster_perf: 0.5191600000000001
Average accuracy over 5 folds for bpr_cluster_perf: 0.51753
